In [5]:
import requests
import pandas as pd
import os
import json

camara_base_url = 'https://dadosabertos.camara.leg.br/api/v2/'

url = f'{camara_base_url}/deputados'
response = requests.get(url)
if not response.ok:
    raise Exception('Nao foi possivel recuperar os dados')

df_deputados = pd.DataFrame().from_dict(json.loads(response.text)['dados'])

df_deputados[['id', 'nome', 'siglaPartido']].to_parquet('data/deputados.parquet')

In [4]:
df_deputados.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliobrunini@camara.leg.br
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


In [7]:
df_parquet = pd.read_parquet('data/deputados.parquet')
df_parquet.head()

,id,nome,siglaPartido
0,220593,Abilio Brunini,PL
1,204379,Acácio Favacho,MDB
2,220714,Adail Filho,REPUBLICANOS
3,221328,Adilson Barroso,PL
4,204560,Adolfo Viana,PSDB


In [8]:
camara_base_url = 'https://dadosabertos.camara.leg.br/api/v2/'

id = 220593

url = f'{camara_base_url}/deputados/{id}/despesas'
response = requests.get(url)
if not response.ok:
    raise Exception('Nao foi possivel recuperar os dados')

response.json()

{'dados': [{'ano': 2024,
   'mes': 8,
   'tipoDespesa': 'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR',
   'codDocumento': 7797837,
   'tipoDocumento': 'Recibos/Outros',
   'codTipoDocumento': 1,
   'dataDocumento': '2024-08-21T00:00:00',
   'numDocumento': '216756',
   'valorDocumento': 149.9,
   'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/3687/2024/7797837.pdf',
   'nomeFornecedor': 'BRASIL ADM E SERVIÇOS DE COBRANÇA',
   'cnpjCpfFornecedor': '33488393000183',
   'valorLiquido': 149.9,
   'valorGlosa': 0.0,
   'numRessarcimento': '',
   'codLote': 2071363,
   'parcela': 0},
  {'ano': 2024,
   'mes': 10,
   'tipoDespesa': 'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR',
   'codDocumento': 7827779,
   'tipoDocumento': 'Nota Fiscal',
   'codTipoDocumento': 0,
   'dataDocumento': '2024-10-05T00:00:00',
   'numDocumento': '218515',
   'valorDocumento': 149.0,
   'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/

In [31]:
import requests
import pandas as pd
import json
from tqdm import tqdm

def fetch_deputados_expenses(id_legislatura='57', ano_despesa='2024', mes_despesa='8', max_itens='100'):
    camara_base_url = 'https://dadosabertos.camara.leg.br/api/v2/'
    list_expenses = []

    df_deputados = pd.read_parquet('data/deputados.parquet')

    def fetch_expenses_for_deputado(id):
        url = f'{camara_base_url}/deputados/{id}/despesas'
        params = {
            'idLegislatura': id_legislatura, 
            'ano': ano_despesa,
            'mes': mes_despesa,
            'itens': int(max_itens),
        }
        while url:
            response = requests.get(url, params=params)
            data = json.loads(response.text)
            df_resp = pd.DataFrame().from_dict(data['dados'])
            df_resp['id'] = id
            list_expenses.append(df_resp)
            url = next((link['href'] for link in data['links'] if link['rel'] == 'next'), None)
            params = None  

    for id in tqdm(df_deputados.id.unique()[:10]):
        fetch_expenses_for_deputado(id)
        
    df_expenses = pd.concat(list_expenses)
    df_expenses = df_expenses.merge(df_deputados, on=['id'])
    df_expenses[[
        'tipoDespesa',
        'dataDocumento',
        'valorDocumento', 
        'nomeFornecedor', 
        'id',
        'nome',
        'siglaPartido'
    ]].to_parquet('data/serie_despesas_diárias_deputados.parquet')

# Example usage:
fetch_deputados_expenses()

100%|██████████| 10/10 [00:01<00:00,  7.12it/s]


In [34]:
df_expenses = pd.read_parquet('data/serie_despesas_diárias_deputados.parquet')
df_expenses.head()[['tipoDespesa', 'dataDocumento', 'valorDocumento', 'nomeFornecedor', 'id', 'nome', 'siglaPartido']]

,tipoDespesa,dataDocumento,valorDocumento,nomeFornecedor,id,nome,siglaPartido
0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,2024-08-21T00:00:00,149.90,BRASIL ADM E SERVIÇOS DE COBRANÇA,220593,Abilio Brunini,PL
1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,2024-08-20T00:00:00,1050.00,BROAD BRASIL LTDA,220593,Abilio Brunini,PL
2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,2024-08-16T00:00:00,659.77,CONDOMÍNIO EDÍFICIL COMERCIAL CONCORDE,220593,Abilio Brunini,PL
3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,2024-08-13T00:00:00,322.62,ENERGISA MATO GROSSO - DISTRIBUIDORA DE ENERGI...,220593,Abilio Brunini,PL
4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,2024-08-02T00:00:00,336.00,Google Cloud Brasil Computação e Serviços de D...,220593,Abilio Brunini,PL


In [52]:
j = json.loads(df_expenses.head(1).to_json())
json.dumps(j, ensure_ascii=False)

'{"tipoDespesa": {"0": "MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"}, "dataDocumento": {"0": "2024-08-21T00:00:00"}, "valorDocumento": {"0": 149.9}, "nomeFornecedor": {"0": "BRASIL ADM E SERVIÇOS DE COBRANÇA"}, "id": {"0": 220593}, "nome": {"0": "Abilio Brunini"}, "siglaPartido": {"0": "PL"}}'

In [60]:
import pandas as pd
import json

def analyze_expenses():
    file_path = 'data/serie_despesas_diárias_deputados.parquet'
    output_path = 'data/insights_despesas_deputados.json'
  
    # Read the parquet file into a pandas DataFrame
    dataframe = pd.read_parquet(file_path)

    # Perform analyses
    max_expense = dataframe.loc[dataframe['valorDocumento'].idxmax()]
    min_expense = dataframe.loc[dataframe['valorDocumento'].idxmin()]
    most_frequent_expense = dataframe['tipoDespesa'].mode()[0]

    # Prepare insights
    insights = {
        "max_expense": {
            "tipoDespesa": str(max_expense['tipoDespesa']),
            "dataDocumento": str(max_expense['dataDocumento']),
            "valorDocumento": float(max_expense['valorDocumento']),
            "nomeFornecedor": str(max_expense['nomeFornecedor']),
            "id": int(max_expense['id']),
            "nome": str(max_expense['nome']),
            "siglaPartido": str(max_expense['siglaPartido'])
        },
        "min_expense": {
            "tipoDespesa": str(min_expense['tipoDespesa']),
            "dataDocumento": str(min_expense['dataDocumento']),
            "valorDocumento": float(min_expense['valorDocumento']),
            "nomeFornecedor": str(min_expense['nomeFornecedor']),
            "id": int(min_expense['id']),
            "nome": str(min_expense['nome']),
            "siglaPartido": str(min_expense['siglaPartido'])
        },
        "most_frequent_expense": str(most_frequent_expense)
    }

    # Write insights to a JSON file
    with open(output_path, 'w') as json_file:
        json.dump(insights, json_file, indent=4, ensure_ascii=False)

    return insights

# Execute the function and print the insights
insights = analyze_expenses()
print(insights)

{'max_expense': {'tipoDespesa': 'LOCAÇÃO OU FRETAMENTO DE AERONAVES', 'dataDocumento': '2024-09-04T00:00:00', 'valorDocumento': 85000.0, 'nomeFornecedor': 'R L FERREIRA - SERVIÇOS E AGENCIAMENTO LTDA', 'id': 220675, 'nome': 'Antônio Doido', 'siglaPartido': 'MDB'}, 'min_expense': {'tipoDespesa': 'PASSAGEM AÉREA - SIGEPA', 'dataDocumento': '2024-08-09T12:00:00', 'valorDocumento': -3113.17, 'nomeFornecedor': 'TAM', 'id': 74253, 'nome': 'Julio Lopes', 'siglaPartido': 'PP'}, 'most_frequent_expense': 'COMBUSTÍVEIS E LUBRIFICANTES.'}
